In [ ]:
#pip install konlpy # 코랩용 모듈 설치문

In [2]:
import pandas as pd
import re
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from tqdm import tqdm
tqdm.pandas()
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [ ]:
df = pd.read_csv('C:/Stocker_Project/Stocker/data/daum_news_samsung_20230301_20240229.csv')
df_stock = pd.read_csv('C:/Stocker_Project/Stocker/data/samsung_stock_20230301_20240228.csv')
df['date'] = pd.to_datetime(df['날짜']).dt.date

# 텍스트 정제 함수
def clean_text(text):
    text = re.sub(r'[^\w\s.,!?%+-]', '', str(text))
    text = re.sub(r'\s+', ' ', text).strip()
    return text

df['clean_title'] = df['뉴스제목'].apply(clean_text) # 뉴스제목 정제

            날짜                                               뉴스제목        date  \
0   2023-03-01                    ‘반도체의 봄’ 언제 오나, 삼성전자 1·2월 적자 3조  2023-03-01   
1   2023-03-01                  ‘갤럭시’→‘삼성’ 일본서 8년 만에 제 이름 찾는 삼성전자  2023-03-01   
2   2023-03-01           삼성전자, 중저가 OLED TV 3종 상반기 추가 출시…점유율 확보 속도  2023-03-01   
3   2023-03-01                       삼성전자·수도권대기환경청, 미세먼지 저감 협약 체결  2023-03-01   
4   2023-03-01                      지난달 국내증시 거래대금 1위는 삼성전자…SM은 4위  2023-03-01   
5   2023-03-01                   삼성전자 ‘30년 60일’ 세계 1위… “생존 근육 단단”  2023-03-01   
6   2023-03-01                     레드햇-삼성전자, 5G 가상화 기지국 솔루션 협력 강화  2023-03-01   
7   2023-03-01                           2월 증시 거래대금 1위 삼성전자…SM 4위  2023-03-01   
8   2023-03-01                     지난달도 개미 `최애`는 삼성전자…거래대금 4위는 SM  2023-03-01   
9   2023-03-01        한종희 삼성전자 부회장, 獨 부총리·정부 인사 만나 부산엑스포 유치 지원 요청  2023-03-01   
10  2023-03-01                                      삼성전자 전시관의 갤럭시  2023-03-01   
11  2023-03-01        삼성전자 '

In [11]:
model_name = "snunlp/KR-FinBERT-SC" # Finbert 모델 지정
tokenizer = AutoTokenizer.from_pretrained(model_name) # 토크나이징
model = AutoModelForSequenceClassification.from_pretrained(model_name)
print(model.config) # 모델 설정 확인
model.to(device) # 모델을 GPU로 이동
label_map = {0: 'negative', 1: 'neutral', 2: 'positive'}

BertConfig {
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "negative",
    "1": "neutral",
    "2": "positive"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "negative": 0,
    "neutral": 1,
    "positive": 2
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "problem_type": "single_label_classification",
  "torch_dtype": "float32",
  "transformers_version": "4.52.4",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 20000
}



In [60]:
# 감성 분석 함수
def predict_sentiment(text):
    inputs = tokenizer(
        text, 
        return_tensors="pt", 
        truncation=True, 
        max_length=128, 
        padding=True
    )
    outputs = model(**inputs)

    probs = torch.nn.functional.softmax(outputs.logits, dim=-1) # 확률값으로 변환
    conf, pred_label = torch.max(probs, dim=1)
    return label_map[pred_label.item()], conf.item()

def get_sentiment_score(text):
    if not text.strip():
        return 0
    label, conf = predict_sentiment(text)

    # 긍부정 확률값 반환
    if label == 'positive':
        return 1 * conf
    elif label == 'negative':
        return -1 * conf
    else:
        return 0


In [ ]:
df['sentiment_score'] = df['clean_title'].progress_apply(get_sentiment_score) # 날짜 기준으로 정렬

daily_sentiment = df.groupby('날짜').agg(
    avg_sentiment=('sentiment_score', 'mean'),
).reset_index()

NameError: name 'df' is not defined

In [65]:
print(df_stock.isnull().sum())
df_stock.dropna(inplace=True) # 결측치 있는 행 삭제

df_stock = df_stock[df_stock['거래량'] > 0] # 거래량 0인 행 삭제

df_stock = df_stock.sort_values('날짜')

print(df_stock[['날짜', '종가', '거래량', '변동률(%)', '상승 여부']].head(10))

날짜        0
종가        0
거래량       0
변동률(%)    0
상승 여부     0
dtype: int64
           날짜     종가       거래량  변동률(%)  상승 여부
0  2023.03.02  60800  13095682    0.00      0
1  2023.03.03  60500  10711405   -0.49      0
2  2023.03.06  61500  13630602    1.65      1
3  2023.03.07  60700  11473280   -1.30      0
4  2023.03.08  60300  14161857   -0.66      0
5  2023.03.09  60100  14334499   -0.33      0
6  2023.03.10  59500  11902471   -1.00      0
7  2023.03.13  60000  12779724    0.84      1
8  2023.03.14  59000  12147346   -1.67      0
9  2023.03.15  59800  10482149    1.36      1


In [ ]:
df_stock = df_stock.loc[:, ~df_stock.columns.duplicated()]

print(df_stock)
print(daily_sentiment)

# 날짜 형식 변호나 & 시간 정규화
daily_sentiment['날짜'] = pd.to_datetime(daily_sentiment['날짜']).dt.normalize()
df_stock['날짜'] = pd.to_datetime(df_stock['날짜']).dt.normalize()

# 날짜 기준으로 두 데이터 내부 조인
df_merged = pd.merge(
    df_stock,
    daily_sentiment,
    on='날짜',
    how='inner',
    suffixes=('_stock', '_sentiment')
)

# 날짜 컬럼 정리 (날짜 겹침)
if '날짜_sentiment' in df_merged.columns:
    df_merged.drop(columns=['날짜_sentiment'], inplace=True)

if '날짜_stock' in df_merged.columns:
    df_merged.rename(columns={'날짜_stock': '날짜'}, inplace=True)

if 'avg_sentiment' in df_merged.columns: # 컬럼명 변경
    df_merged.rename(columns={'avg_sentiment': '감성 점수'}, inplace=True)

df_merged['감성 점수'] = df_merged['감성 점수'].fillna(0) # 결측값 0으로 채우기

df_merged = df_merged.sort_values('날짜')
df_merged['변동률(%)'] = df_merged['변동률(%)'].fillna(0)


In [ ]:
print(df_merged.head())
df_merged['감성점수'] = df_merged['감성점수'].round(2)

df_merged.to_csv('data_preprocessing.csv', index=False)


          날짜     종가       거래량  변동률(%)  상승 여부  감성점수
0 2023-03-02  60800  13095682    0.00      0  0.55
1 2023-03-03  60500  10711405   -0.49      0  0.27
2 2023-03-06  61500  13630602    1.65      1  0.50
3 2023-03-07  60700  11473280   -1.30      0  0.16
4 2023-03-08  60300  14161857   -0.66      0 -0.01
